In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [4]:
# print tensorflow version
import tensorflow as tf
print(tf.__version__)

2024-03-20 21:01:43.720173: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 21:01:43.720261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 21:01:43.747349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 21:01:43.803122: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 21:01:44.607045: W tensorflow/compiler/tf2

2.15.0


In [3]:
df = pd.read_csv('/home/fuck/Desktop/projects/arabic/ar_reviews_100k.tsv', sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [4]:
df.shape

(99999, 2)

In [5]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [6]:
df = df.rename(columns={'label': 'Sentiment', 'text': 'Tweet'})

In [7]:
df['Sentiment'].value_counts()

Sentiment
Positive    33333
Mixed       33333
Negative    33333
Name: count, dtype: int64

In [13]:
df.duplicated().sum()

0

In [14]:
# remove punctuation
remove_punctuations = [
    "/::\)","/::","(-_-)","(*_*)","(>_<)",":)",";)",":P","xD","-_-","#","(>_<)","...",".",",",";",":","!","?","'","অ�", "অাবার", "।","?",
    "\"","-","_","/","\\","|","{","}","[","]","(",")","<",">","@","#","$","%","^","&","*","~","`","+","=","0","1","2","3","4","5","6","7","8","9","৳","০",
    "১","২","৩","৪","৫","৬","৭","৮","৯","\n","\t","\r","\f","\v","\u00C0-\u017F","\u2000-\u206F","\u25A0-\u25FF","\u2600-\u26FF","\u2B00-\u2BFF","\u3000-\u303F",
    "\uFB00-\uFB4F","\uFE00-\uFE0F","\uFE30-\uFE4F","\u1F600-\u1F64F","\u1F300-\u1F5FF","\u1F680-\u1F6FF","\u1F1E0-\u1F1FF","\u2600-\u26FF","\u2700-\u27BF",
    "\u1F300-\u1F5FF","\u1F900-\u1F9FF","\u1F600-\u1F64F","\u1F680-\u1F6FF","\u1F1E0-\u1F1FF","\u1F600-\u1F64F",
]
# reset index of the dataframe
df.reset_index(inplace=True)

In [15]:
for i in range(len(df)):
    text = df.loc[i,'Text']
    for punctuation in remove_punctuations:
        text = text.replace(punctuation,' ')
    df.loc[i,'Text'] = text

In [16]:
import re

# remove emoji
def remove_emoji(text):
    emoji_pattern = re.compile(
        "["u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", text)

In [17]:
# remove emoji
for i in range(len(df)):
    text = df.loc[i,'Text']
    text = remove_emoji(text)
    df.loc[i,'Text'] = text

In [18]:
# remove english character
def remove_english_character(text):
    english_character = re.compile("[a-zA-Z]+")
    return english_character.sub(r"", text)

In [20]:
# remove english character
for i in range(len(df)):
    text = df.loc[i,'Text']
    text = remove_english_character(text)
    df.loc[i,'Text'] = text

In [19]:
# remove extra space
def remove_extra_space(text):
    extra_space = re.compile("\s+")
    return extra_space.sub(r" ", text)

In [21]:
# remove extra space
for i in range(len(df)):
    text = df.loc[i,'Text']
    text = remove_extra_space(text)
    df.loc[i,'Text'] = text

In [8]:
df.head()

,Sentiment,Tweet
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [9]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import TFBertForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.layers import Embedding

2024-03-22 04:10:54.501684: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 04:10:54.501724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 04:10:54.502523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 04:10:54.508031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 04:10:55.140872: W tensorflow/compiler/tf2

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
import tensorflow as tf

In [12]:
# Tokenize the text
text = df['Tweet'].values
max_length = 128
Xids = np.zeros((len(text), max_length))
Xmask = np.zeros((len(text), max_length))

for i, sentence in enumerate(text):
    tokens = tokenizer.encode_plus(sentence, max_length=max_length, truncation=True, padding='max_length', add_special_tokens=True, return_tensors='tf')
    Xids[i, :], Xmask[i, :] = tokens['input_ids'], tokens['attention_mask']

2024-03-22 04:10:59.748086: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 04:10:59.773408: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 04:10:59.773608: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [13]:
le = LabelEncoder()
df['Sentiment'] = le.fit_transform(df['Sentiment'])

labels = to_categorical(df['Sentiment'], num_classes=3)

df.head()
y = df['Sentiment']

# Split the data into training, testing and validation sets
Xids_train, Xids_test, y_train, y_test = train_test_split(Xids, y, test_size=0.2, random_state=42)
Xids_train, Xids_val, y_train, y_val = train_test_split(Xids_train, y_train, test_size=0.2, random_state=42)

In [14]:
from tensorflow.keras.layers import Layer, Input, Dense, LSTM, GRU, Concatenate, Dropout, Embedding
from tensorflow.keras.models import Model

In [15]:
# create attention embedder with q, k, v
class AttentionEmbedder(tf.keras.Model):
    def __init__(self, units):
        super(AttentionEmbedder, self).__init__()
        self.Wq = tf.keras.layers.Dense(units)
        self.Wk = tf.keras.layers.Dense(units)
        self.Wv = tf.keras.layers.Dense(units)
        self.units = units

    def call(self, inputs):
        q = self.Wq(inputs)
        k = self.Wk(inputs)
        v = self.Wv(inputs)
        return q, k, v

In [16]:
# create attention layer
class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def call(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

In [17]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Creating the attention weights variables
        self.dense_lstm = Dense(1, activation='softmax')
        self.dense_gru = Dense(1, activation='softmax')
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        lstm_output, gru_output = inputs
        
        # Attention mechanism for LSTM output
        attention_weights_lstm = self.dense_lstm(lstm_output)
        context_vector_lstm = tf.reduce_sum(attention_weights_lstm * lstm_output, axis=1)

        # Attention mechanism for GRU output
        attention_weights_gru = self.dense_gru(gru_output)
        context_vector_gru = tf.reduce_sum(attention_weights_gru * gru_output, axis=1)

        # Concatenate context vectors
        context_vector = Concatenate()([context_vector_lstm, context_vector_gru])
        
        return context_vector

In [18]:
# import cnn and global
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Input, Embedding, LSTM, Conv1D, GRU, GlobalMaxPooling1D, concatenate, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Model

In [19]:
attention_units=64

In [26]:
def custom_encoder_with_attention(input_length, vocab_size, embedding_dim, attention_units):
    input_word_ids = Input(shape=(input_length,), dtype=tf.int32, name="input_word_ids")

    # Embedding layer
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_word_ids)

    # attention embedder
    attention_embedder = AttentionEmbedder(attention_units)
    q, k, v = attention_embedder(embedding_layer)
    

    # Bi-LSTM layers
    bilstm_output = Bidirectional(LSTM(128, return_sequences=True))(embedding_layer)
    bilstm_output = Bidirectional(LSTM(128, return_sequences=True))(bilstm_output)

    # Bi-GRU layers
    bigru_output = Bidirectional(GRU(128, return_sequences=True))(bilstm_output)
    bigru_output = Bidirectional(GRU(128, return_sequences=True))(bigru_output)

    # Attention layer with attention weights
    attention_layer = AttentionLayer()
    
    # Attention mechanism
    context_vector = attention_layer([bilstm_output, bigru_output])
    
    # Output
    encoder_output = Dense(128, activation='relu')(context_vector)
    encoder_output = Dropout(0.2)(encoder_output)
    encoder_output = Dense(64, activation='relu')(encoder_output)
    encoder_output = Dropout(0.2)(encoder_output)
    output = Dense(3, activation='softmax')(encoder_output)

    # Create model
    model = Model(inputs=input_word_ids, outputs=output)
    return model


In [27]:
input_length = max_length
vocab_size = tokenizer.vocab_size + 1
embedding_dim = 100
attention_units = 64

In [28]:
# Create and compile the model
encoder_model_with_attention = custom_encoder_with_attention(input_length, vocab_size, embedding_dim, attention_units)
encoder_model_with_attention.compile(optimizer="adam",
                                     loss="sparse_categorical_crossentropy",
                                     metrics=['accuracy'])

In [79]:
# For example:
Xids_train = Xids_train.reshape(-1, max_length)
Xids_val = Xids_val.reshape(-1, max_length)
Xids_test = Xids_test.reshape(-1, max_length)

In [65]:
encoder_model_with_attention.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [80]:
encoder_model_with_attention.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 embedding_8 (Embedding)     (None, 128, 100)             3052300   ['input_word_ids[0][0]']      
                                                                                                  
 attention_embedder_3 (Atte  ((None, 128, 64),            19392     ['embedding_8[0][0]']         
 ntionEmbedder)               (None, 128, 64),                                                    
                              (None, 128, 64))                                              

In [81]:
epochs = 10
batch_size = 32
history = encoder_model_with_attention.fit(
    {'input_word_ids': Xids_train},
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=({'input_word_ids': Xids_val}, y_val)
)

Epoch 1/10
2000/2000 [==============================] - 335s 163ms/step - loss: 1.1251 - accuracy: 0.3322 - val_loss: 1.0986 - val_accuracy: 0.3370
Epoch 2/10
2000/2000 [==============================] - 308s 154ms/step - loss: 1.0987 - accuracy: 0.3308 - val_loss: 1.0986 - val_accuracy: 0.3370
Epoch 3/10
2000/2000 [==============================] - 303s 152ms/step - loss: 1.0987 - accuracy: 0.3306 - val_loss: 1.0987 - val_accuracy: 0.3315
Epoch 4/10
2000/2000 [==============================] - 304s 152ms/step - loss: 1.0987 - accuracy: 0.3320 - val_loss: 1.0986 - val_accuracy: 0.3370
Epoch 5/10
2000/2000 [==============================] - 303s 152ms/step - loss: 1.0987 - accuracy: 0.3295 - val_loss: 1.0986 - val_accuracy: 0.3315
Epoch 6/10
2000/2000 [==============================] - 303s 152ms/step - loss: 1.0987 - accuracy: 0.3356 - val_loss: 1.0988 - val_accuracy: 0.3315
Epoch 7/10
2000/2000 [==============================] - 302s 151ms/step - loss: 1.0987 - accuracy: 0.3315 - val_

In [23]:
encoder_model_with_attention.compile(optimizer="nadam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [24]:
epochs = 10
batch_size = 32
history = encoder_model_with_attention.fit(
    {'input_word_ids': Xids_train},
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=({'input_word_ids': Xids_val}, y_val)
)

Epoch 1/10
2000/2000 [==============================] - 161s 77ms/step - loss: 0.6388 - accuracy: 0.7032 - val_loss: 0.7398 - val_accuracy: 0.6532
Epoch 2/10
  42/2000 [..............................] - ETA: 2:15 - loss: 0.6106 - accuracy: 0.7299

KeyboardInterrupt: 

In [ ]:
class SelfAttentionEmbedder(Layer):
    def __init__(self, d_model, num_heads):
        super(SelfAttentionEmbedder, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // self.num_heads

        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)

        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        q = self.wq(inputs)
        k = self.wk(inputs)
        v = self.wv(inputs)

        batch_size = tf.shape(inputs)[0]

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(q, k, v)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)
        return output

    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

In [ ]:
class MultiHeadSelfAttention(Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
        return output, attention_weights

    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights